In [33]:
import pandas as pd

In [34]:
# xls = pd.ExcelFile('Online Retail.xlsx')
# df1 = pd.read_excel(xls)
# df1.to_csv('dataset.csv')

In [35]:
df = pd.read_csv('dataset.csv')
df.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [36]:
df.isna().sum()

Unnamed: 0          0
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [37]:
df = df.drop(columns=['Unnamed: 0','CustomerID'])

In [38]:
df.fillna("", inplace=True)
df.isna().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
Country        0
dtype: int64

In [ ]:
# Группируем товары по транзакциям
transactions = df.groupby('InvoiceNo')['Description'].apply(lambda x: set(x)).tolist()


In [40]:
# Проверка данных
print(f"Всего транзакций: {len(transactions)}")
print("Пример транзакций:")
for i, transaction in enumerate(transactions[:5]):
    print(f"Транзакция {i + 1}: {transaction}")

Всего транзакций: 25900
Пример транзакций:
Транзакция 1: {'WHITE METAL LANTERN', 'WHITE HANGING HEART T-LIGHT HOLDER', 'SET 7 BABUSHKA NESTING BOXES', 'CREAM CUPID HEARTS COAT HANGER', 'RED WOOLLY HOTTIE WHITE HEART.', 'GLASS STAR FROSTED T-LIGHT HOLDER', 'KNITTED UNION FLAG HOT WATER BOTTLE'}
Транзакция 2: {'HAND WARMER RED POLKA DOT', 'HAND WARMER UNION JACK'}
Транзакция 3: {'IVORY KNITTED MUG COSY ', 'BOX OF VINTAGE ALPHABET BLOCKS', "POPPY'S PLAYHOUSE KITCHEN", 'LOVE BUILDING BLOCK WORD', "POPPY'S PLAYHOUSE BEDROOM ", 'FELTCRAFT PRINCESS CHARLOTTE DOLL', 'DOORMAT NEW ENGLAND', 'BOX OF 6 ASSORTED COLOUR TEASPOONS', 'HOME BUILDING BLOCK WORD', 'BOX OF VINTAGE JIGSAW BLOCKS ', 'RECIPE BOX WITH METAL HEART', 'ASSORTED COLOUR BIRD ORNAMENT'}
Транзакция 4: {'BLUE COAT RACK PARIS FASHION', 'YELLOW COAT RACK PARIS FASHION', 'JAM MAKING SET WITH JARS', 'RED COAT RACK PARIS FASHION'}
Транзакция 5: {'BATH BUILDING BLOCK WORD'}


как работает априори?

1. Генерируем кандидатов длины 1
    - найти все уникальные
    - для каждого вычислить поддержку
    - отобрать, где поддержка > минсуппорт
2. генерируем кандидатов длины  k>1
    - на основе часто встречающихся k-1 генерируем кандидатов длины k
    - обхединяем наборы длин k-1 и k-2
    - {A, B} и {A, C} -> {A,B,C}
3. проверяем поддержку кандидатов k
4. повторить пока не найдем все

In [41]:
class AprioriMyself:
    def __init__(self, dataset, min_support):
        self.dataset = dataset
        self.min_support = min_support
        self.frequent_itemsets = []

    def _generate_candidates(self, itemsets, length):
        """Генерация кандидатов длины length."""
        candidates = set()
        for itemset1 in itemsets:
            for itemset2 in itemsets:
                union = itemset1.union(itemset2)
                if len(union) == length:
                    candidates.add(union)
        return candidates

    def fit(self, debug:bool=False):
        """Обучение модели и поиск часто встречающихся наборов."""
        items = set()
        for transaction in self.dataset:
            items.update(transaction)

        length = 1
        candidates = [frozenset([item]) for item in items]

        while candidates:
            if debug:
                print(f"Текущая длина кандидатов: {length}")
                print(f"Количество кандидатов: {len(candidates)}")

            candidate_counts = np.zeros(len(candidates))
            for transaction in self.dataset:
                for i, candidate in enumerate(candidates):
                    if candidate.issubset(transaction):
                        candidate_counts[i] += 1

            frequent_candidates = [
                candidate for i, candidate in enumerate(candidates)
                if candidate_counts[i] / len(self.dataset) >= self.min_support
            ]
            if debug:
                print(f"Найдено {len(frequent_candidates)} часто встречающихся наборов длины {length}.")

            self.frequent_itemsets.extend(frequent_candidates)
            candidates = list(self._generate_candidates(frequent_candidates, length + 1))
            length += 1

    def get_frequent_itemsets(self):
        """Возвращает часто встречающиеся наборы."""
        return self.frequent_itemsets

In [42]:
apriori_model = AprioriMyself(df, min_support=0.001)
apriori_model.fit(True)
print("Часто встречающиеся наборы (Apriori):", apriori_model.get_frequent_itemsets())

Текущая длина кандидатов: 1
Количество кандидатов: 23
Найдено 0 часто встречающихся наборов длины 1.
Часто встречающиеся наборы (Apriori): []


In [45]:
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder

# Преобразуем данные в бинарный формат
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Запуск Apriori
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
print(frequent_itemsets)

       support                                           itemsets
0     0.056139                                                 ()
1     0.010502                 ( SET 2 TEA TOWELS I LOVE LONDON )
2     0.012587                           (10 COLOUR SPACEBOY PEN)
3     0.013784                    (12 PENCIL SMALL TUBE WOODLAND)
4     0.014131              (12 PENCILS SMALL TUBE RED RETROSPOT)
...        ...                                                ...
1034  0.012664  (RED RETROSPOT CHARLOTTE BAG, STRAWBERRY CHARL...
1035  0.010232  (SET/20 RED RETROSPOT PAPER NAPKINS , SET/6 RE...
1036  0.010077  (CHARLOTTE BAG SUKI DESIGN, RED RETROSPOT CHAR...
1037  0.011699  (GREEN REGENCY TEACUP AND SAUCER, REGENCY CAKE...
1038  0.010386  (JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO BAG ...

[1039 rows x 2 columns]


Основные шаги алгоритма FP-Growth:

1. Построение FP-дерева (Frequent Pattern Tree):
    - Первый проход по данным: 
        - алгоритм проходит по всему датасету и подсчитывает частоту каждого элемента.
        - элементы, которые встречаются реже, чем заданный minsupport, удаляются, а оставшиеся сортируются в порядке убывания частоты.
    - Второй проход по данным: 
        - Каждая транзакция из базы данных преобразуется в путь в дереве, начиная с корня. 
        - Узлы дерева представляют элементы, а пути от корня до узлов представляют наборы элементов, которые встречаются вместе в транзакциях. 
        - Если несколько транзакций содержат одинаковые наборы элементов, то соответствующие узлы объединяются, и их счетчики увеличиваются.

2. Извлечение частых наборов:
   - алгоритм извлекает частые наборы элементов, используя дерево. Это делается путем "роста" частых паттернов из листьев к корню дерева, используя conditional FP-trees, которые строятся для каждого элемента, начиная с наименее частых.
   - для каждого элемента, алгоритм строит его условное FP-дерево, которое содержит только те элементы, которые встречаются вместе с данным элементом. 
   - из этого дерева извлекаются все возможные комбинации элементов, которые образуют частые наборы.

In [46]:
class FPGrowthMyself:
    def __init__(self, dataset, min_support):
        """
        Инициализация класса FPGrowthMyself.

        :param dataset: Входные данные (список транзакций, где каждая транзакция — это набор элементов).
        :param min_support: Минимальная поддержка (число от 0 до 1).
        """
        self.dataset = dataset
        self.min_support = min_support
        self.frequent_itemsets = []

    class FPTreeNode:
        """Вспомогательный класс для узла FP-Tree."""
        def init(self, item, count, parent):
            self.item = item
            self.count = count
            self.parent = parent
            self.children = {}
            self.next = None

    def _build_fp_tree(self):
        """Построение FP-Tree."""
        item_counts = {}
        for transaction in self.dataset:
            for item in transaction:
                item_counts[item] = item_counts.get(item, 0) + 1

        self.frequent_items = {item for item, count in item_counts.items() if count / len(self.dataset) >= self.min_support}

        if not self.frequent_items:
            return None, None

        root = self.FPTreeNode(None, None, None)
        header_table = {}

        for transaction in self.dataset:
            filtered_items = [item for item in transaction if item in self.frequent_items]
            filtered_items.sort(key=lambda x: item_counts[x], reverse=True)

            current_node = root
            for item in filtered_items:
                if item in current_node.children:
                    current_node = current_node.children[item]
                    current_node.count += 1
                else:
                    new_node = self.FPTreeNode(item, 1, current_node)
                    current_node.children[item] = new_node
                    current_node = new_node

                    if item in header_table:
                        last_node = header_table[item]
                        while last_node.next:
                            last_node = last_node.next
                        last_node.next = new_node
                    else:
                        header_table[item] = new_node

        return root, header_table

    def _mine_fp_tree(self, header_table, prefix):
        """Рекурсивный поиск часто встречающихся наборов."""
        for item in header_table:
            new_prefix = prefix.copy()
            new_prefix.add(item)
            self.frequent_itemsets.append(new_prefix)

            conditional_pattern_base = []
            node = header_table[item]
            while node:
                path = []
                parent = node.parent
                while parent.item:
                    path.append(parent.item)
                    parent = parent.parent
                if path:
                    conditional_pattern_base.append((path, node.count))
                node = node.next

            conditional_tree, conditional_header = self._build_fp_tree([pattern for pattern, count in conditional_pattern_base for _ in range(count)])
            if conditional_header:
                self._mine_fp_tree(conditional_header, new_prefix)

    def fit(self):
        """Обучение модели и поиск часто встречающихся наборов."""
        root, header_table = self._build_fp_tree()
        if header_table:
            self._mine_fp_tree(header_table, set())

    def get_frequent_itemsets(self):
        """Возвращает часто встречающиеся наборы."""
        return self.frequent_itemsets

# Пример использования
fp_growth_model = FPGrowthMyself(transactions, min_support=0.3)
fp_growth_model.fit()
print("Часто встречающиеся наборы (FP-Growth):", fp_growth_model.get_frequent_itemsets())

Часто встречающиеся наборы (FP-Growth): []


In [47]:
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder

# Запуск Apriori
frequent_itemsets = fpgrowth(df, min_support=0.01, use_colnames=True)
print(frequent_itemsets)

       support                                           itemsets
0     0.088880               (WHITE HANGING HEART T-LIGHT HOLDER)
1     0.017876              (KNITTED UNION FLAG HOT WATER BOTTLE)
2     0.016795                   (RED WOOLLY HOTTIE WHITE HEART.)
3     0.014865                     (SET 7 BABUSHKA NESTING BOXES)
4     0.012201                              (WHITE METAL LANTERN)
...        ...                                                ...
1034  0.013900  (JUMBO BAG VINTAGE DOILY , JUMBO BAG RED RETRO...
1035  0.010927  (JUMBO BAG VINTAGE DOILY , JUMBO BAG VINTAGE L...
1036  0.010772  (LUNCH BAG VINTAGE DOILY , LUNCH BAG RED RETRO...
1037  0.010541  (JUMBO BAG VINTAGE DOILY , LUNCH BAG VINTAGE D...
1038  0.010463  (CHOCOLATE HOT WATER BOTTLE, HOT WATER BOTTLE ...

[1039 rows x 2 columns]
